In [1]:
require('nn')
require('hdf5')
require('optim')

In [2]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [3]:
--changes index to account for window size
function changeIndex(X, window_size, nfeatures)
    for i=1,X:size(2) do
        X:select(2,i):add((i-1)*(nfeatures))
    end
    return X
end

function lookup(X, Y, win_size, C, K)
    W = torch.zeros(C, win_size*(K))
    for i=1,X:size(1) do
        for j=1,X:size(2) do
            local row = Y[i]
            local col = X[i][j]
            W[row][col] = W[row][col] + 1
        end
    end
    return W
end

In [7]:
function CBM(fc, Y, alpha, V)
    --calculate log posterior
    --local cc = torch.zeros(1, V)
    --local clp = torch.zeros(1, V)

    --collapse column wise so Nx1
    --local hist = torch.histc(Y:double(), 10001)
    --cc:add(torch.reshape(hist, 1, 10001))
    --THIS IS ALPHA
    --smoothing
    --cc:add(alpha)
    fc:add(alpha)
    local scc = fc:sum(2)
    
    fc:log()
    scc:log()
    
    scc:expand(scc, scc:size(1), fc:size(2))
    
    --calculate log prior
    --collapse down so 1x1
    --local total = cc:sum(2)
    --cc:div(total[1][1])
    --cc:log()
    --clp:csub(math.log(Y:size(2)))
    
    return fc:csub(scc)--, cc
end

In [10]:
function predict(X, W, b)
  local jll = torch.zeros(X:size(1), W:size(1))
  for i = 1, X:size(1) do
    jll[i] = W:index(2, X[i]):sum(2)
    --jll[i]:add(b)
  end
  return jll
end

In [15]:
function kaggle_perplexity(jll, Y)
    local matrix = torch.zeros(Y:size(1), Y:size(2))
    for i=1, Y:size(1) do
        for j=1, Y:size(2) do
            idx = Y[i][j]
            matrix[i][j] = (jll[i][idx] * -1)
        end
    end
    return matrix:cdiv(torch.expand((torch.sum(matrix, 2)), Y:size(1), matrix:size(2)))
end

function valid_perplexity(jll, Y_correct, Y)
    jll:cdiv(torch.expand((torch.sum(jll, 2)), Y:size(1), jll:size(2)))
    perp = 0
    for i=1, Y_correct:size(1) do
        idx = Y[i][Y_correct[i]]
        perp = (torch.log(jll[i][idx])) + perp
    end
    print(torch.exp(perp/Y:size(1) * -1))
end

function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [4]:
X_train = changeIndex(X_train, 5, nfeatures)
X_valid = changeIndex(X_valid, 5, nfeatures)
X_test = changeIndex(X_test, 5, nfeatures)

In [5]:
W = lookup(X_train, Y_train, window_size, 10001, nfeatures)

In [8]:
W_ = CBM(W, Y_train, 1, 10001)

In [11]:
jll = predict(X_test, W_, b)

In [5]:
valid_perplexity(jll, q_valid, )

 9987
[torch.LongTensor of size 1]



In [9]:
W:size()

 10001
 50010
[torch.LongStorage of size 2]



In [12]:
W_ = W_:cdiv(torch.expand(W_:sum(2), W_:size(1), W_:size(2)))